# Bayesian Optimization with Ax

### Setup

#### Import Libraries

This script uses the Ax Bayesian Optimization model to try to find the best parameters for optimizing a battery for the University of Utah Chemical Engineering Car Club. There is a 2D example of this script in the AxDemonstration notebook. 

In [3]:
from ax.service.ax_client import AxClient
import pandas as pd
from ax.service.utils.instantiation import ObjectiveProperties
from IPython.display import clear_output
from time import sleep

### Train the Model 

Using the collected data, train the Ax model and use it to suggest a new set of parameters to test. 

The output of this code will be a set of the parameters that you need to test next.

Run the code below to get a list of parameters to try. 

In [4]:
# Read the CSV file
df = pd.read_csv('../Datasets/ObservedData.csv')

ax_client = AxClient(verbose_logging=False)

# Define the parameters based on your CSV columns
parameters = [
    {"name": "SizeOfCell", "type": "choice", "values": [1.53, 1.8, 2.0]},
    {"name": "ManganeseGraphiteRatio", "type": "range", "bounds": [1, 3.5]},
    {"name": "LayersOfPaper", "type": "choice", "values": [1, 2]},
    {"name": "Chloride", "type": "range", "bounds": [0.9, 1.5]}
]

# Create the experiment
ax_client.create_experiment(
    name="battery_optimization",
    parameters=parameters,
    objectives={"y": ObjectiveProperties(minimize=False)}
)

# Attach existing trials from the CSV
for i, row in df.iterrows():
    parameters = {"SizeOfCell": row["Size of the Cell"], "ManganeseGraphiteRatio": row["Manganese to Graphite Ratio"], "LayersOfPaper": int(row["Layers of Filter Paper"]), "Chloride": row["3M Ammonium Chloride Saturation (mL)"]}
    objective_value = row['Power (W)']
    ax_client.attach_trial(parameters)
    ax_client.complete_trial(trial_index=i, raw_data={"y": objective_value})

# Get next set of parameters to try
parameters, trial_index = ax_client.get_next_trial()

# Compute x3 based on the generated x2
parameters["Graphite"] = 1 - parameters["Manganese"]

# Assume the total weight is 15 grams
total_weight = 15.0  # in grams

# weight percent of manganese dioxide
manganese_weight_percent = (parameters["ManganeseGraphiteRatio"])/(parameters["ManganeseGraphiteRatio"] + 1)
graphite_weight_percent = 1 - manganese_weight_percent
manganese_weight = total_weight * manganese_weight_percent
graphite_weight = total_weight * graphite_weight_percent

# Constants
molarity = 3.0  # Molarity in moles per liter
molar_mass_ammonium_chloride = 53.49  # Molar mass of NH4Cl in g/mol

# Given volume of the solution in milliliters, convert to liters
volume_liters = parameters["Chloride"] / 1000  # Convert mL to L

# Calculate moles of ammonium chloride needed
moles_ammonium_chloride = molarity * volume_liters

# Convert moles to grams
grams_ammonium_chloride = (total_weight/1000) * 3 * molar_mass_ammonium_chloride

clear_output()
sleep(0.2)

# Print the next set of parameters to try, including weights in grams
print("Next set of parameters to try:")
print(f"Size of the Cell: {parameters['SizeOfCell']:.3f} cm diameter")
print(f"Manganese Dioxide: {manganese_weight:.2f} g ({parameters['Manganese']*100:.1f}%)")
print(f"Graphite: {graphite_weight:.2f} g ({parameters['Graphite']*100:.1f}%)")
print(f"Ratio of Manganese Dioxide to Graphite: {parameters['ManganeseGraphiteRatio']:.2f}:1")
print(f"Layers of Filter Paper: {parameters['LayersOfPaper']} layers")
print(f"Water: {parameters['Chloride']*15:.3f} ml ({parameters['Chloride']:.2f}% Saturation)")
print(f"Dry Ammonium Chloride required: {(parameters['Chloride']*grams_ammonium_chloride):.2f} g")

[INFO 09-06 15:21:16] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter SizeOfCell. If that is not the expected value type, you can explicitly specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
/opt/miniconda3/envs/chemcar/lib/python3.11/site-packages/ax/service/utils/instantiation.py:244: AxParameterWarning: `is_ordered` is not specified for `ChoiceParameter` "SizeOfCell". Defaulting to `True`  since the parameter is not of type string.. To override this behavior (or avoid this warning), specify `is_ordered` during `ChoiceParameter` construction. Note that choice parameters with exactly 2 choices are always considered ordered and that the user-supplied `is_ordered` has no effect in this particular case.
  return ChoiceParameter(
/opt/miniconda3/envs/chemcar/lib/python3.11/site-packages/ax/service/utils/instantiation.py:244: AxParameterWarning: `sort_values` is not specified for `ChoiceParameter` "SizeOfCell". Defaulting to 

AssertionError: Values in `bounds` not of the same type and no `value_type` was explicitly specified; cannot infer value type for parameter ManganeseGraphiteRatio.

### Log the Data

Run the code below to get the input prompt to log the data. This number is the average voltage recorded from the battery using the suggested parameters. 

In [ ]:
# Here you would typically run your experiment with these parameters
# and get a result. For this example, let's just use a dummy result
result = float(input("Input the tested variable"))  # Replace this with your actual experimental result

# Complete the trial with the result
ax_client.complete_trial(trial_index=trial_index, raw_data={"y": result})

### Save the Data

Run the cell below to save the new parameters and observed value to the csv file. This will allow the model to incorperate the new data into its future fittings.

In [ ]:

# round parameters to 3 decimal places
parameters = {key: round(value, 3) for key, value in parameters.items()}

# Prepare the new row as a DataFrame
new_row = pd.DataFrame([{
    "Size of the Cell": parameters['SizeOfCell'],
    "Manganese to Graphite Ratio": parameters['ManganeseGraphiteRatio'],
    "Layers of Filter Paper": parameters['LayersOfPaper'],
    "3M Ammonium Chloride Saturation (mL)": parameters['Chloride'],
    "Power (W)": result
}])

# Concatenate the new row to the existing DataFrame
df = pd.concat([df, new_row], ignore_index=True)

# Save the updated dataframe to a CSV file
df.to_csv('../Datasets/ObservedData.csv', index=False)

### Print the top parameters found so far

In [ ]:
# Get the best parameters
best_parameters, metrics = ax_client.get_best_parameters()

# Print the best parameters to 3 decimal places
print("\nBest parameters:")
for key, value in best_parameters.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

print("\nBest target value:")
print(metrics[0]['y'])


Best parameters:
SizeOfCell: 1.530
Manganese: 0.542
LayersOfPaper: 1
Chloride: 1.425

Best target value:
110.0
